# Label DP SGD

This notebook walks through how to train a model to recognize hand written
digits using label differentially private gradient decent and the MNIST dataset.
In this setting, one party has the images and the other party has the labels.
They would like to collaborate to train a model without revealing their data.

Before starting, install the tf-shell package.

```bash
pip install tf-shell
```

First, import some modules and set up tf-shell. The parameters are for the SHELL
encryption library, which tf-shell uses, and mostly depend on the multiplicative
depth of the computation to be performed. This example performs back
propagation, thus the multiplicative depth is determined by the number of
layers. For more information, see [SHELL](https://github.com/google/shell).

In [1]:
import time
import os
from datetime import datetime
import tensorflow as tf
import keras
import numpy as np
import tf_shell
import tf_shell_ml

2024-09-29 06:18:05.175618: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-29 06:18:05.198038: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Set a default batch size (must be less that the chosen ciphertext ring degree
# so anything less than 2**10 is fine). This will be used for validation, but
# for training using autocontext (as below) the batch size is determined by the
# ciphertext parameters.
batch_size = 2**10

# Setup the dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train, x_test = np.reshape(x_train, (-1, 784)), np.reshape(x_test, (-1, 784))
x_train, x_test = x_train / np.float32(255.0), x_test / np.float32(255.0)
y_train, y_test = tf.one_hot(y_train, 10), tf.one_hot(y_test, 10)

epochs = 1
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_dataset.shuffle(buffer_size=2048)
    .batch(batch_size, drop_remainder=True)
    .repeat(count=epochs)
)

val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(batch_size, drop_remainder=True)

In [3]:
# Turn on the shell optimizer to use autocontext.
tf_shell.enable_optimization()
use_fast_reduce_sum = True

m = tf_shell_ml.TfShellSequential(
    [
        tf_shell_ml.ShellDense(
            64,
            activation=tf_shell_ml.relu,
            activation_deriv=tf_shell_ml.relu_deriv,
            use_fast_reduce_sum=use_fast_reduce_sum,
        ),
        tf_shell_ml.ShellDense(
            10,
            activation=tf.nn.softmax,
            use_fast_reduce_sum=use_fast_reduce_sum,
        ),
    ],
    lambda: tf_shell.create_autocontext64(
        log2_cleartext_sz=12,
        scaling_factor=3,
        noise_offset_log2=68,
    ),
    True,
)

m.compile(
    shell_loss=tf_shell_ml.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.1),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.CategoricalAccuracy()],
)

train_datset = m.set_dataset_batching(train_dataset)

# m.build([batch_size, 784])  # do not build if using autoparams
# m(train_dataset)
# m.summary()


Final parameters:
log_n: 12
t: 65537
qs: 288230376151760897 288230376152137729 


In [4]:
# Set up tensorboard logging.
stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
fast_str = "-fast" if use_fast_reduce_sum else ""
logdir = os.path.abspath("") + f"/tflogs/dp-sgd{fast_str}-{stamp}"

tboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir,
                                                 histogram_freq = 1,
                                                 profile_batch = '500,520')

history = m.fit(train_dataset, epochs=1, validation_data=val_dataset, callbacks = [tboard_callback])

2024-09-29 06:18:37.535177: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-09-29 06:18:37.535200: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-09-29 06:18:37.535262: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.


Final parameters:
log_n: 12
t: 65537
qs: 288230376151760897 288230376152137729 
58/58 [==============================] - 1160s 20s/step - num_slots: 4096.0000 - val_categorical_accuracy: 0.1018


In [5]:
m.summary()

Model: "tf_shell_sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shell_dense (ShellDense)    multiple                  50176     
                                                                 
 shell_dense_1 (ShellDense)  multiple                  640       
                                                                 
Total params: 50816 (198.50 KB)
Trainable params: 50816 (198.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
